In [1]:
import json
from us import states
from pymongo import MongoClient

#Connect to Mongo DB
myclient = MongoClient("mongodb://localhost:27017/")
  
#Census Data
db = myclient["CensusData"]

collections = db.list_collection_names()
print ("collections:", collections, "\n")

def get_state_data(state_fips):
    


collections: ['counties_geocoding', 'states', 'counties', 'census_datadump', 'popincome_data', 'insurance_data', 'counties_sub', 'deathrate_data'] 



In [12]:
popincome_data = db.popincome_data.find()

counties_collected = []
# List to store the outputed dictionaries
counties_data = []
for popincome in popincome_data:

    insurance_data = db.insurance_data.find(({"state": popincome["state"],
                                          "county": popincome["county"],
                                          "time":popincome["Year"]}),{'_id': False})

    #If this is the first time encountering a particular county, creates a new dictionary for it, stores data
    if popincome["Name"] not in counties_collected:
        counties_collected.append(popincome["Name"])
        json = {}
        json["Name"] = popincome["Name"],
        json["state"] = popincome["state"]
        json["county"] = popincome["county"]
        information = {}
        information["Population"] = popincome["Population"],
        information["Median Age"] = popincome["Median Age"]
        information["Per Capita Income"] = popincome["Per Capita Income"]
        information["Poverty Count"] = popincome["Poverty Count"]
        for insurance_info in insurance_data:
            information["Insured"] = insurance_data["NIC_PT"]
            information["Uninsured"] = insurance_data["NUI_PT"]
        # information['Educational Attainment'] = popincome['Educational Attainment']
        json[popincome["Year"]] = information
        counties_data.append(json)
        
    # If it has arleady encounterd the county, appends the standing entry and stores the new year's data
    else:
        location = counties_collected.index(popincome["Name"])
        json = counties_data[location]
        information = {}
        information["Population"] = popincome["Population"],
        information["Median Age"] = popincome["Median Age"]
        information["Per Capita Income"] = popincome["Per Capita Income"]
        information["Poverty Count"] = popincome["Poverty Count"]
        # information['Educational Attainment'] = popincome['Educational Attainment']
        json[popincome["Year"]] = information

In [3]:
county_ids = []
for county in counties_data:
    county_id = county["state"]+county["county"]
    county_ids.append(county_id)

In [7]:
counties_list = db.counties.find()
counties_geojson_data = db.counties_geocoding.find({},{"_id":False})

counties_geojson_final=[]

failed_county_lookup=[]
for county_geojson in counties_geojson_data:
    lookup_county = county_geojson["properties"]['STATE'] + county_geojson["properties"]['COUNTY']
    try:
        #Attempts to locate any county which has the same county_id in the censusJson List
        location = county_ids.index(str(lookup_county))
        matching_county_data = counties_data[location]
        county_geojson["properties"][2018] = matching_county_data[2018]
        county_geojson["properties"][2019] = matching_county_data[2019]
        county_geojson["properties"][2020] = matching_county_data[2020]
        counties_geojson_final.append(county_geojson)
    except Exception:
        #If it cannot locate it stores the id for this county in the fail list
        failed_county_lookup.append(lookup_county)

In [8]:
for county_geojson in counties_geojson_final:
    #Generate the countyID as above for each GeoJson
    county_geojson_check = county_geojson["properties"]['STATE'] + county_geojson["properties"]['COUNTY']
    if county_geojson_check in failed_county_lookup:
        counties_geojson_final.remove(county_geojson)

In [ ]:
# # Get counties
# states_l = db.counties.aggregate([{"$group": {"_id": {"STATENAME" :"$STATENAME","STATEFP":"$STATEFP"}}}])
# #https://www2.census.gov/geo/docs/reference/codes/

# # for state in states_l:
# #     print(state)


In [ ]:
# county_geocoding_list = []
# # for each county build a json with state FIPS id
# for state in states_l:
#     cursor_geocoding = db.counties_geocoding.find({"properties.STATE": states_l["STATENAME"],
#                                                       "properties.COUNTY": states_l["STATEFP"]},{'_id': False})
#     # query_geocoding = db.counties_geocoding.find(
#     #                 {"$and": [{ "properties" : { "$elemMatch" :{ "STATE": state_fips.fips}} },
#     #                 { "properties" : { "$elemMatch" :{ "COUNTY": county["county_fips"]}}d
#     #                 }]})
#     for county_geocoding in cursor_geocoding:
#         county_geocoding_list[0].append(county_geocoding)
#     print (county_geocoding_list)
# #     state_fips = us.states[county_geo_record["state_id"]].fips
# #     print(county_geo_record["state_id"],state_fips)